# Figure 8
Compatible with PyKonal Version 0.2.0

In [3]:
%matplotlib ipympl

import matplotlib.pyplot as plt
import numpy as np
import os
import pkg_resources
import pykonal
import scipy.ndimage

In [4]:
from scipy.optimize import differential_evolution

In [5]:
import bokeh.plotting as bp
from bokeh.io import output_notebook, curdoc, output_file
output_notebook()

Loading BokehJS ...

In [6]:
import json
import pandas as pd
#from pandas.io.json import json_normalize

In [7]:
with open("/Users/sak/Desktop/MELT/GZDS/12-JAN-2019/42_water.json") as f:
    j = json.load(f)
    df=pd.json_normalize(j['picks'], sep='_')
    df_water = df.loc[:,['offset', 'autopicks_picks10_peakTime', 'autopicks_picks_peakTime']]
    df_water = df_water.rename({'offset': 'x', 'autopicks_picks10_peakTime':'t10', 'autopicks_picks_peakTime': 't'}, axis='columns')
with open("/Users/sak/Desktop/MELT/GZDS/12-JAN-2019/42_ice.json") as f:
    j = json.load(f)
    df_ice=pd.json_normalize(j['picks'], sep='_')
    df_ice = df_ice.loc[:,['offset', 'autopicks_picks10_peakTime', 'autopicks_picks_peakTime']]
    df_ice = df_ice.rename({'offset': 'x', 'autopicks_picks10_peakTime':'t10', 'autopicks_picks_peakTime': 't'}, axis='columns')

In [276]:
df_water.to_csv("42_water.csv", index=False, columns=["x", "t"])
df_ice.to_csv("42_ice.csv", index=False, columns=["x", "t"])

In [284]:
iceobs = np.genfromtxt("42_ice.csv", delimiter=",", skip_header=1) 
waterobs = np.genfromtxt("42_water.csv", delimiter=",", skip_header=1) 
print(iceobs[:,1])

[0.3265  0.32625 0.326   0.3255  0.325   0.32475 0.3245  0.324   0.3235
 0.32375 0.323   0.32275 0.3225  0.32225 0.3225  0.322   0.32125 0.321
 0.321   0.321   0.321   0.321   0.321   0.3215  0.321   0.32075 0.32075
 0.3215  0.32125 0.3215  0.3215  0.322   0.32225 0.32275 0.3225  0.323
 0.323   0.3235  0.324   0.3245  0.32425 0.3245  0.325   0.32475 0.32525
 0.32575 0.3265  0.327  ]


In [206]:
def make_nodes(vel, z, alpha):
    """"for geom in vel, find nodes for a layer that has depth z at x=0, alpha in deg."""
    npts = vel.npts
    nx=npts[0]
    nz=npts[1]
    ny=npts[2]
    d = vel.node_intervals
    dx=d[0]
    dz=d[1]
    dy=d[2]
    mn = vel.min_coords
    mnx=mn[0]
    mnz=mn[1]
    mny=mn[2]
    #print(nx, nz, ny, dx, dz, dy, mnx, mnz, mny)
    lay = np.ndarray(nx, dtype=int)
    for ix in range(nx):
        idx = int(z/dz) + int((ix*dx+mnx)*np.tan(alpha * np.pi/180)/dz)
        if idx >= nz:
            idx = nz-1
        if idx < 0:
            idx = 0
        lay[ix] = int(idx)
        #print(ix, idx)
        #vel.values[ix, idx:nz] = 1.5
    return lay

In [221]:
def make_vel(vel, zi, alphai, zw, alphaw):
    "ice shelf geom with vi=3.8 0->zi  and vw=1.5  from zi->zw vsed=4.5 below"
    npts = vel.npts
    nx=npts[0]
    nz=npts[1]
    ny=npts[2]
    
    vel.values[0:nx, 0:nz] = 3.8
    icelay = make_nodes(vel, zi, alphai)
    waterlay = make_nodes(vel, zw, alphaw)
    
    for i in range(len(icelay)):
        if waterlay[i] <  icelay[i]:
            waterlay[i] =  icelay[i]

    for ix in range(nx):
        #idx = int(zi/dz) + int(ix*dx*np.tan(alphai)/dz)
        #print(ix, idx)
        ili = icelay[ix] #ice bottom index
        wli = waterlay[ix] #water bottom index
        # don't let water bottom rise above ice bottom
        if  wli>ili:
            vel.values[ix, ili:wli] = 1.5
            vel.values[ix, wli:nz] = 4.5
        else:
            vel.values[ix, ili:nz] = 4.5
    #vel.values[0:nx-1, 60:70] = 1.5
    #vel.values[0:nx-1,  71:nz-1] = 4.5
    return vel, icelay, waterlay

In [203]:
def calc_refl(velocity, shotloc_x, shotloc_z, layer_idxs):
    """calculate the reflection tt in velocity grid with given shotloc off layer_idxs
    velocity is a pykonal velocity obj
    shotloc_[xz] are positions relative to velocity.min_coords
    layer_idxs is an array of depth indices (z) for each x node that denotes the reflection surface
    """
    solver_dg = pykonal.EikonalSolver(coord_sys="cartesian")
    solver_dg.vv.min_coords = velocity.min_coords
    solver_dg.vv.node_intervals = velocity.node_intervals
    solver_dg.vv.npts = velocity.npts
    solver_dg.vv.values = velocity.values

    #shotloc = 2.56 # km
    src_idx = (int((shotloc_x - velocity.min_coords[0])/velocity.node_intervals[0]), int(shotloc_z/velocity.node_intervals[1]), 0)
    solver_dg.tt.values[src_idx] = 0
    solver_dg.unknown[src_idx] = False
    solver_dg.trial.push(*src_idx)
    solver_dg.solve()

    solver_ug = pykonal.EikonalSolver(coord_sys="cartesian")
    solver_ug.vv.min_coords = solver_dg.vv.min_coords
    solver_ug.vv.node_intervals = solver_dg.vv.node_intervals
    solver_ug.vv.npts = solver_dg.vv.npts
    solver_ug.vv.values = solver_dg.vv.values

    for ix in range(solver_ug.tt.npts[0]):
        #idx = (ix, solver_ug.tt.npts[1]-1, 0)
        idx = (ix, layer_idxs[ix], 0)
        solver_ug.tt.values[idx] = solver_dg.tt.values[idx]
        #print(idx, solver_dg.tt.values[idx])
        solver_ug.unknown[idx] = False
        solver_ug.trial.push(*idx)
    solver_ug.solve()
    
    ofs = np.arange(solver_ug.vv.min_coords[0],  solver_ug.vv.)
    
    return solver_ug.tt.values[:,0,0]

In [257]:
def tt_err(refl, velocity, shotloc_x, shotloc_z, obs_x, obs_t,  disp=False):
    "given a reflector (depth refl[0] at x=0, dip refl[1]), calculate tt  through velocity model  from  shotloc and  compare to obs"
    refl_z=refl[0]
    refl_ang=refl[1]
    layer_idxs = make_nodes(velocity, refl_z, refl_ang)
    tt_mod = calc_refl(velocity, shotloc_x, shotloc_z, layer_idxs)
    err = 0
    for x,t in zip(obs_x, obs_t):
        (ix, iz, iy) = get_idx(velocity, (x/1000, 0, 0))
        #print(ix, x)
        modt = tt_mod[ix]
        diff = t-modt
        if disp:
            print(x, t, ix, modt, diff)
        err = err + diff**2
    print("z=", refl_z, "ang=", refl_ang, "err=", err)
    return err
def tt_err_waterbot(refl, zi, alphai, velocity, shotloc_x, shotloc_z, obs_x, obs_t,  disp=False):
    "given a water bott refl (depth refl[0] at x=0, dip refl[1]), calculate tt through velocity model from  shotloc and  compare to obs"
    refl_z=refl[0]
    refl_ang=refl[1]
    v, icelay, waterlay = make_vel(velocity, zi, alphai, refl_z, refl_ang) #make_nodes(velocity, refl_z, refl_ang)
    tt_mod = calc_refl(v, shotloc_x, shotloc_z, waterlay)
    err = 0
    for x,t in zip(obs_x, obs_t):
        (ix, iz, iy) = get_idx(v, (x/1000, 0, 0))
        #print(ix, x)
        modt = tt_mod[ix]
        diff = t-modt
        if disp:
            print(x, t, ix, modt, diff)
        err = err + diff**2
    print("z=", refl_z, "ang=", refl_ang, "err=", err)
    return err

In [52]:
def ice_mod():
    velocity = pykonal.fields.ScalarField3D(coord_sys="cartesian")
    velocity.min_coords = -2.56, 0, 0
    velocity.node_intervals = 0.01, 0.001, 0.01
    velocity.npts = 512, 1024, 1
    #velocity.values = 3.8*np.ones(velocity.npts)
    #velocity.values[0:511,60:70] = 1.5
    #velocity.values[0:511,71:127] = 4.5
    npts = velocity.npts
    nx=npts[0]
    nz=npts[1]
    ny=npts[2]
    
    velocity.values[0:nx, 0:nz] = 3.8
    #icelay = make_nodes(vel, zi, alphai)
    #waterlay = make_nodes(vel, zw, alphaw)

    #for ix in range(nx):
        #idx = int(zi/dz) + int(ix*dx*np.tan(alphai)/dz)
        #print(ix, idx)
    #    ili = icelay[ix]
    #    wli = waterlay[ix]
    #    vel.values[ix, ili:wli] = 1.5
    #    vel.values[ix, wli:nz] = 4.5
    #vel.values[0:nx-1, 60:70] = 1.5
    #vel.values[0:nx-1,  71:nz-1] = 4.5
    return velocity

def water_mod(zi, alphai):
    velocity = pykonal.fields.ScalarField3D(coord_sys="cartesian")
    velocity.min_coords = -2.56, 0, 0
    velocity.node_intervals = 0.01, 0.001, 0.01
    velocity.npts = 512, 1024, 1
    #velocity.values = 3.8*np.ones(velocity.npts)
    #velocity.values[0:511,60:70] = 1.5
    #velocity.values[0:511,71:127] = 4.5
    npts = velocity.npts
    nx=npts[0]
    nz=npts[1]
    ny=npts[2]
    
    velocity.values[0:nx, 0:nz] = 3.8
    icelay = make_nodes(velocity, zi, alphai)
    #waterlay = make_nodes(vel, zw, alphaw)

    for ix in range(nx):
        #idx = int(zi/dz) + int(ix*dx*np.tan(alphai)/dz)
        #print(ix, idx)
        ili = icelay[ix]
    #    wli = waterlay[ix]
        velocity.values[ix, ili:nz] = 1.5
    #    vel.values[ix, wli:nz] = 4.5
    #vel.values[0:nx-1, 60:70] = 1.5
    #vel.values[0:nx-1,  71:nz-1] = 4.5
    return velocity
    
    #velocity = make_vel(velocity, 0.610, 0*np.pi/180,  1, 0*np.pi/180)
    #icelay = make_nodes(velocity, 0.61, 0*np.pi/180)
    #waterlay = make_nodes(velocity, 0.66, -4.2*np.pi/180)
#scipy.ndimage.gaussian_filter(20. * np.random.randn(*velocity.npts) + 6., 10)

In [258]:
icemod = ice_mod()
#icelay = make_nodes(icemod, 0.61, 0.55)
watermod = water_mod(0.61, 0.55)
vel, icelay, waterlay = make_vel(icemod, 0.61, 0.55, 0.66, -3)
#print(watermod.values[256,600:700])

In [228]:
print(waterlay)

[964 963 961 960 959 958 956 955 954 953 952 950 949 948 947 945 944 943
 942 940 939 938 937 936 934 933 932 931 929 928 927 926 925 923 922 921
 920 918 917 916 915 913 912 911 910 909 907 906 905 904 902 901 900 899
 898 896 895 894 893 891 890 889 888 886 885 884 883 882 880 879 878 877
 875 874 873 872 871 869 868 867 866 864 863 862 861 859 858 857 856 855
 853 852 851 850 848 847 846 845 843 842 841 840 839 837 836 835 834 832
 831 830 829 828 826 825 824 823 821 820 819 818 816 815 814 813 812 810
 809 808 807 805 804 803 802 801 799 798 797 796 794 793 792 791 789 788
 787 786 785 783 782 781 780 778 777 776 775 774 772 771 770 769 767 766
 765 764 762 761 760 759 758 756 755 754 753 751 750 749 748 746 745 744
 743 742 740 739 738 737 735 734 733 732 731 729 728 727 726 724 723 722
 721 719 718 717 716 715 713 712 711 710 708 707 706 705 704 702 701 700
 699 697 696 695 694 692 691 690 689 688 686 685 684 683 681 680 679 678
 677 675 674 673 672 670 669 668 667 665 664 663 66

In [259]:
#waterlay = make_nodes(watermod, 0.655, -7)
xx=calc_refl(vel,  0, 0, waterlay)
for i in range(len(xx)):
    print(i, xx[i])

0 0.8726119866557254
1 0.8700983248143445
2 0.8675857498142985
3 0.865074278977529
4 0.8625639300888658
5 0.8600547214295088
6 0.8575466717830265
7 0.8550398004577248
8 0.8525341272995709
9 0.8500296727039517
10 0.8475264576430808
11 0.8450245036929818
12 0.8425238330340924
13 0.8400244685179591
14 0.8375264336559548
15 0.8350297526767075
16 0.8325344505538689
17 0.8300405530801689
18 0.8275480868697793
19 0.8250570794195148
20 0.8225675591791661
21 0.8200795555930931
22 0.8175930991560241
23 0.8151082214963704
24 0.8126249554307645
25 0.8101433350257143
26 0.8076633956495208
27 0.805185174034844
28 0.8027087082764043
29 0.8002340378407098
30 0.79776120351175
31 0.7952902472626056
32 0.792821212018044
33 0.790354141329932
34 0.787889078860038
35 0.7854260676672581
36 0.7829651492600951
37 0.7805063623690687
38 0.7780497414093129
39 0.7755953146619832
40 0.7731431022158168
41 0.7706931137190358
42 0.7682453461429136
43 0.765799781790927
44 0.763356386781209
45 0.7609151103823594
46 0.75

In [260]:
tt_err_waterbot([0.66, -3], 0.61, 0.55, watermod, 0,  0, df_water.x,   df_water.t, True)

235 0.3795 279 0.38149321052716184 -0.0019932105271618372
225 0.37925000000000003 278 0.38140118144091445 -0.002151181440914418
215 0.3785 277 0.38132649710146266 -0.002826497101462655
205 0.37825000000000003 276 0.38140569109274847 -0.003155691092748436
195 0.37875000000000003 275 0.3815130926457991 -0.002763092645799048
185 0.37875000000000003 274 0.38165636443129775 -0.0029063644312977144
175 0.3785 273 0.3818369059502066 -0.003336905950206581
165 0.37825000000000003 272 0.38203936357718077 -0.0037893635771807377
155 0.378 271 0.38221374149430437 -0.004213741494304368
145 0.37875000000000003 270 0.38222207138203895 -0.0034720713820389215
135 0.37875000000000003 269 0.38229489395625976 -0.0035448939562597315
125 0.37625000000000003 268 0.38238008784876576 -0.006130087848765731
115 0.379 267 0.38246517120767204 -0.0034651712076720353
105 0.3795 266 0.3825660043130496 -0.0030660043130495973
95 0.37975000000000003 265 0.3826994190467034 -0.002949419046703372
85 0.37975000000000003 264 0

0.0003667763101407922

In [137]:
differential_evolution(tt_err, bounds=[(0.4, 0.8),  (-8, 8)], mutation=(0.5,1.5), args=(icemod, 0, 0, df_ice.x, df_ice.t))

z= 0.4385099339544547 ang= -7.534698276276737 err= 0.3843322725647518
z= 0.7320320250094579 ang= -5.037459398995928 err= 0.19674840745641534
z= 0.7062135867977728 ang= 5.956005584915376 err= 0.12196632317797
z= 0.677325920323849 ang= -1.7567763065725037 err= 0.059570072793119586
z= 0.48509607293585366 ang= 1.883933078593607 err= 0.20303353212656566
z= 0.4031029146770335 ang= -2.2878580282183654 err= 0.5561768542694331
z= 0.45790159532471864 ang= -0.7065176305340941 err= 0.30418619485530674
z= 0.44056544650601065 ang= 4.815747591130567 err= 0.37545907004128926
z= 0.5525621236459473 ang= -2.9525418213578165 err= 0.043518062912501466
z= 0.6589015748413158 ang= 7.888150758339309 err= 0.030693641164502253
z= 0.5208430189971045 ang= -6.946406044750578 err= 0.1051402442387269
z= 0.6185930671584192 ang= -1.4566091621916515 err= 0.0009204909649014948
z= 0.5362262924259797 ang= -4.43432647978854 err= 0.07099340743070155
z= 0.6906367039931874 ang= -4.00770322759983 err= 0.08480183190349619
z= 0.5

z= 0.6072425252953428 ang= -5.997884298053001 err= 9.707546780497607e-05
z= 0.6540547328463227 ang= -4.38674347784602 err= 0.02582436407908025
z= 0.608887613873712 ang= -3.0213183520166202 err= 4.079578028435969e-05
z= 0.6022368373691159 ang= 0.2093653876968169 err= 0.0007717412781063356
z= 0.6144051332588275 ang= -1.5968006563645467 err= 0.0002538394172251624
z= 0.5956629907442601 ang= 0.18060086108796014 err= 0.0028170928970591333
z= 0.6549995856615468 ang= 2.871351765640437 err= 0.02582436407908025
z= 0.6523078573235417 ang= -1.2586918439664174 err= 0.02354748396461668
z= 0.6206135515172146 ang= -2.5214150622342775 err= 0.0014112437402502103
z= 0.6203713749289134 ang= -3.8662399518062696 err= 0.0014112437402502103
z= 0.7352129387873304 ang= -5.582226114178219 err= 0.20652416122064374
z= 0.6202466821298167 ang= -3.885715473939177 err= 0.0014112437402502103
z= 0.5856965533128315 ang= -0.44267463035094146 err= 0.007965703267858427
z= 0.6567243867543808 ang= -3.81666747405904 err= 0.028

z= 0.6105219988306253 ang= 5.493749629241428 err= 6.904009451751087e-06
z= 0.617387540496209 ang= 4.463897149382911 err= 0.0007144738090425545
z= 0.6233598075665673 ang= 7.411673394887165 err= 0.002344193534772243
z= 0.6177023177111733 ang= -6.075366116287091 err= 0.0007144738090425545
z= 0.5998383395183493 ang= 7.174804024062759 err= 0.0014910899680829154
z= 0.6135482432923388 ang= -1.4603844503581165 err= 0.00015276140376188808
z= 0.6046651219378198 ang= 2.7798322644597455 err= 0.00042322861107337023
z= 0.6148014740783003 ang= 5.613838926561888 err= 0.0002538394172251624
z= 0.608887613873712 ang= 1.2653717350632832 err= 4.079578028435969e-05
z= 0.6148364457346084 ang= -1.0262661697570454 err= 0.0002538394172251624
z= 0.6071414316146886 ang= 3.976562086831313 err= 9.707546780497607e-05
z= 0.611150315193072 ang= 4.686236235082957 err= 2.9295408961617185e-05
z= 0.6136542038532532 ang= 5.815125873924172 err= 0.00015276140376188808
z= 0.6156517236878917 ang= 0.15918082868341443 err= 0.000

z= 0.6102827282517432 ang= 5.67990914781128 err= 6.904009451751087e-06
z= 0.6086001540899864 ang= 2.1612993625116363 err= 4.079578028435969e-05
z= 0.6097926581570874 ang= 0.8479972266417999 err= 1.0738045171787815e-05
z= 0.6106888148316253 ang= 0.2167313979926746 err= 6.904009451751087e-06
z= 0.6104893482673245 ang= 3.085317697892041 err= 6.904009451751087e-06
z= 0.6105188181138613 ang= 0.9719586623987642 err= 6.904009451751087e-06
z= 0.6107276786027633 ang= 1.2220770566546086 err= 6.904009451751087e-06
z= 0.6111247030933734 ang= -0.6349828589589528 err= 2.9295408961617185e-05
z= 0.610309666581647 ang= 2.9624360633660753 err= 6.904009451751087e-06
z= 0.6104442225963546 ang= 1.615163287681419 err= 6.904009451751087e-06
z= 0.6104724245071298 ang= 0.07323331829777224 err= 6.904009451751087e-06
z= 0.6103069055246708 ang= 6.520732805136834 err= 6.904009451751087e-06
z= 0.6103860472371088 ang= 3.6167539624620257 err= 6.904009451751087e-06
z= 0.6104157762534017 ang= 4.041345137676355 err= 6.9

     fun: 6.904009451751087e-06
 message: 'Optimization terminated successfully.'
    nfev: 453
     nit: 14
 success: True
       x: array([0.61073967, 5.03035577])

In [261]:
differential_evolution(tt_err_waterbot, bounds=[(0.61, 0.8),  (-8, 8)], args=(0.61, 0.55, watermod, 0, 0, df_water.x, df_water.t))

z= 0.6659801413036601 ang= -5.616365938366983 err= 0.0005316497994027442
z= 0.7302097548447596 ang= 3.799985637016455 err= 0.42767779149788715
z= 0.7814068903708987 ang= -6.6590832414334145 err= 1.0782553793913257
z= 0.6458303800668337 ang= -0.9634679818968168 err= 0.013586667742195899
z= 0.738798334056831 ang= 3.012082652974623 err= 0.5435118976554967
z= 0.7518006876022991 ang= 7.649679734889942 err= 0.5642330809957639
z= 0.6264699361576258 ang= -3.3276385869787335 err= 0.09398558926534042
z= 0.6947588990691815 ang= -2.9391908577094954 err= 0.10635664391258705
z= 0.671314011084985 ang= 7.088137552426771 err= 0.015463493956943365
z= 0.6316439958887899 ang= 2.6654130553822295 err= 0.06722950044302228
z= 0.6782581452262075 ang= 5.178559037679239 err= 0.02995882754972273
z= 0.616222711430571 ang= 4.635972853744827 err= 0.16580441322742961
z= 0.7096193443117156 ang= -1.5220017651971327 err= 0.22912515191853697
z= 0.6209750907692723 ang= 0.7214353357203453 err= 0.11737237632462515
z= 0.7660

z= 0.6682724453135538 ang= -4.598474124958204 err= 0.0034928537868385684
z= 0.6541276970735715 ang= -7.550335413273782 err= 0.025830357187426715
z= 0.6684903313365333 ang= -0.6359333131775848 err= 0.012512306913780887
z= 0.6706272565704959 ang= 0.11857812131741774 err= 0.017473195629657677
z= 0.6889492142935253 ang= 0.4634711810645573 err= 0.08630949587346777
z= 0.6664512179652832 ang= -3.790583812890544 err= 0.003169059459752449
z= 0.6768446280043955 ang= -4.234604229684901 err= 0.019304640879906604
z= 0.6877900987553242 ang= -0.9634679818968168 err= 0.07820388426558735
z= 0.652773947672058 ang= 3.012082652974623 err= 0.008939955784737242
z= 0.6778126725954444 ang= -3.906137057076249 err= 0.023747352849256054
z= 0.6711222971123989 ang= -3.0592541095923664 err= 0.012834370981622356
z= 0.6692279966975043 ang= -7.311391582508732 err= 0.001734992624865912
z= 0.654113893234199 ang= -7.066498227035471 err= 0.020675467768317076
z= 0.6594706475228882 ang= 3.101118086500101 err= 0.005670315901

z= 0.6569301566333731 ang= -4.945092116856268 err= 0.003878460527381569
z= 0.6590409375188049 ang= -1.381112184452471 err= 0.0014577638319209214
z= 0.6604591830619796 ang= -3.7569665002006607 err= 8.451973236317799e-05
z= 0.6616223664493401 ang= -4.173775920460605 err= 6.99344598496551e-05
z= 0.6659801413036601 ang= -4.482501504207054 err= 0.0011588174650471105
z= 0.6648122002157878 ang= -7.882956260503246 err= 0.008231669984145745
z= 0.669317848145255 ang= -3.7161017158469587 err= 0.0070669294713642085
z= 0.6697196516080345 ang= -5.856582511831029 err= 0.0018616848744058023
z= 0.653923755339031 ang= -3.0888503604934687 err= 0.0031384090968866202
z= 0.6663060278704073 ang= -6.533337583181064 err= 0.001079299075460767
z= 0.6570468935327338 ang= -2.7852537903059282 err= 0.0005608871887874448
z= 0.6701020699561123 ang= -6.228235911446358 err= 0.0018307940000576936
z= 0.6580384367756972 ang= -2.3428660511168786 err= 0.0005442947535088237
z= 0.6588162541882281 ang= -0.7314386816705607 err= 

z= 0.6588372154367692 ang= -3.2906083066285037 err= 0.0002768786045934666
z= 0.661205156342081 ang= -3.452717133977437 err= 0.00031680762322904205
z= 0.6616366911288827 ang= -3.326065161995115 err= 0.0004157569454964169
z= 0.6626139888839694 ang= -3.89052629288849 err= 0.0003334168826779842
z= 0.661405421637542 ang= -3.8136689229099288 err= 0.00017447109359632136
z= 0.6608254257165692 ang= -3.619088968910992 err= 0.00010264635936356048
z= 0.6598923362275115 ang= -3.1644859756732178 err= 0.00018236146382996196
z= 0.6604159164296187 ang= -3.762364664758988 err= 8.451973236317799e-05
z= 0.6609895411517648 ang= -3.871039898036087 err= 6.698061107568251e-05
z= 0.6603824889267567 ang= -3.488895752644461 err= 0.00013970288564613104
z= 0.6624037477739803 ang= -4.464539832513749 err= 0.00011689840084694382
z= 0.6620635997156524 ang= -4.2246669497248375 err= 0.0001682011767120057
z= 0.6608655346221275 ang= -3.979730753078236 err= 8.348750362731744e-05
z= 0.6623721856038245 ang= -3.04688692563604

z= 0.6614497262671885 ang= -4.41619515835481 err= 8.32924035021512e-05
z= 0.6624183618649385 ang= -4.3658764102138665 err= 0.0001317980209450064
z= 0.6625299174808651 ang= -3.8338701780317255 err= 0.0003342711557388784
z= 0.6613815538105641 ang= -4.092326717202909 err= 7.09429275212585e-05
z= 0.6615253950558408 ang= -4.158841007835758 err= 7.229688056135902e-05
z= 0.6612715103889748 ang= -4.401408956640501 err= 8.32924035021512e-05
z= 0.6608119393438254 ang= -3.8919905933965655 err= 6.932177710182252e-05
z= 0.661515437723299 ang= -4.157958129099215 err= 7.229688056135902e-05
z= 0.6622325942374105 ang= -4.480095998014243 err= 0.00012662436981351945
z= 0.6623439687216889 ang= -3.8635823098045456 err= 0.0003342711557388784
z= 0.6605661307637984 ang= -4.070352783277749 err= 9.019079011890179e-05
z= 0.6619325836498681 ang= -4.312683840590148 err= 6.914914223467647e-05
z= 0.6614751290414173 ang= -4.029708592773686 err= 8.864937913089427e-05
z= 0.6618539636848173 ang= -3.9914077279615405 err=

z= 0.6612494171555982 ang= -4.00380717276405 err= 9.1570383952407e-05
z= 0.6611161215124104 ang= -4.221085661110385 err= 6.45064422749625e-05
z= 0.660738408391058 ang= -4.38159215870113 err= 0.0002074756060869402
z= 0.6610950524036807 ang= -4.196931282762908 err= 6.449309447145869e-05
z= 0.6616622904037112 ang= -4.311663077975101 err= 6.914914223467647e-05
z= 0.661429613343181 ang= -3.8996083520829705 err= 0.00011645319859038679
z= 0.6609440465317749 ang= -4.115997846491623 err= 0.00010155267331026267
z= 0.6608956094234361 ang= -4.28230847134906 err= 0.00014910439729254156
z= 0.6607828639043676 ang= -4.216683328527308 err= 0.00013041816125910287
z= 0.662142486758069 ang= -4.1948100922857705 err= 0.00016890212700096117
z= 0.6608956094234361 ang= -4.28230847134906 err= 0.00014910439729254156
z= 0.6614728503404274 ang= -4.290482147293458 err= 6.914914223467647e-05
z= 0.660727695888711 ang= -4.5452265920504535 err= 0.00031357167086662566
z= 0.6621331741537406 ang= -4.475703053765558 err= 0

z= 0.6606566857143431 ang= -3.8690623936316344 err= 6.698061107568251e-05
z= 0.6600478363857465 ang= -3.8944187313040706 err= 6.932177710182252e-05
z= 0.6611161001560949 ang= -4.148501254368496 err= 7.229688056135902e-05
z= 0.6611772198736057 ang= -3.849817033312072 err= 0.00011568884369170314
z= 0.661410638001508 ang= -4.558808837731241 err= 0.00013444576307096735
z= 0.6614540029393591 ang= -4.191570718709077 err= 6.993477575562181e-05
z= 0.6607070205004898 ang= -4.220333069763137 err= 0.00013041816125910287
z= 0.6613989262435902 ang= -3.8874940387975947 err= 0.00011568884369170314
z= 0.6601614052232673 ang= -4.221085661110385 err= 0.00013041816125910287
z= 0.660738408391058 ang= -4.091570450250616 err= 0.00010155267331026267
z= 0.661061081932555 ang= -4.196931282762908 err= 6.449309447145869e-05
z= 0.6605390322662129 ang= -4.205315534608324 err= 0.00012973374410340602
z= 0.6607764948784571 ang= -4.200407896219156 err= 0.00012973374410340602
z= 0.6607736521152685 ang= -4.1160702770014

z= 0.6613442994092932 ang= -4.478202389790635 err= 0.00012114884368976796
z= 0.6607428585510059 ang= -4.200867554483938 err= 0.00012973374410340602
z= 0.6611652909768546 ang= -4.195421175856666 err= 6.449309447145869e-05
z= 0.6612121858838427 ang= -4.200793815481761 err= 6.449309447145869e-05
z= 0.6610747601522475 ang= -3.858840923408412 err= 0.00011568884369170314
z= 0.6610760824184182 ang= -4.196978391737206 err= 6.449309447145869e-05
z= 0.6615309525522248 ang= -4.191909198226602 err= 6.993477575562181e-05
z= 0.6608357221647648 ang= -4.196892702836262 err= 0.00012973374410340602
z= 0.6607336703225601 ang= -4.199273740340907 err= 0.00012973374410340602
z= 0.660959696749876 ang= -4.19471297621793 err= 0.00012973374410340602
z= 0.6610719255077474 ang= -4.200738959794558 err= 6.449309447145869e-05
z= 0.6610750622168922 ang= -4.199350180352665 err= 6.449309447145869e-05
z= 0.6613704637388488 ang= -4.461653860645531 err= 0.00010280776403022791
z= 0.6613044916803028 ang= -4.20771417390352 e

     fun: 6.449309447145869e-05
 message: 'Optimization terminated successfully.'
    nfev: 813
     nit: 26
 success: True
       x: array([ 0.66108821, -4.19693128])

In [13]:
def get_idx(velocity, coord):
    """given velocity, calculate idices for coord(3-tuple)"""
    d = velocity.node_intervals
    dx=d[0]
    dz=d[1]
    dy=d[2]
    mn = velocity.min_coords
    mnx=mn[0]
    mnz=mn[1]
    mny=mn[2]
    ix  = int((coord[0] - mnx)/dx)
    iz  = int((coord[1] - mnz)/dz)
    iy  = int((coord[2] - mny)/dy)
    return (ix, iz, iy)

In [262]:
velocity = pykonal.fields.ScalarField3D(coord_sys="cartesian")
velocity.min_coords = -2.56, 0, 0
velocity.node_intervals = 0.01, 0.001, 0.01
velocity.npts = 512, 1024, 1
#velocity.values = 3.8*np.ones(velocity.npts)
#velocity.values[0:511,60:70] = 1.5
#velocity.values[0:511,71:127] = 4.5
velocity,icelay,waterlay = make_vel(velocity, 0.610, 0.55,  0.661, -4.2)
#icelay = make_nodes(velocity, 0.61, 0)
#waterlay = make_nodes(velocity, 0.655, -7, water=True)
#scipy.ndimage.gaussian_filter(20. * np.random.randn(*velocity.npts) + 6., 10)
solver_dg = pykonal.EikonalSolver(coord_sys="cartesian")
solver_dg.vv.min_coords = velocity.min_coords
solver_dg.vv.node_intervals = velocity.node_intervals
solver_dg.vv.npts = velocity.npts
solver_dg.vv.values = velocity.values

shotloc_x = 2.56 # km
shotloc_z  = 0
src_idx = (int(shotloc_x/velocity.node_intervals[0]), int(shotloc_z/velocity.node_intervals[1]), 0)
solver_dg.tt.values[src_idx] = 0
solver_dg.unknown[src_idx] = False
solver_dg.trial.push(*src_idx)
solver_dg.solve()


solver_ug = pykonal.EikonalSolver(coord_sys="cartesian")
solver_ug.vv.min_coords = solver_dg.vv.min_coords
solver_ug.vv.node_intervals = solver_dg.vv.node_intervals
solver_ug.vv.npts = solver_dg.vv.npts
solver_ug.vv.values = solver_dg.vv.values

for ix in range(solver_ug.tt.npts[0]):
    #idx = (ix, solver_ug.tt.npts[1]-1, 0)
    idx = (ix, waterlay[ix], 0)
    solver_ug.tt.values[idx] = solver_dg.tt.values[idx]
    solver_ug.unknown[idx] = False
    solver_ug.trial.push(*idx)
solver_ug.solve()

1

In [263]:
plt.close("all")
fig = plt.figure(figsize=(6, 2.5))

ax1 = fig.add_subplot(2, 1, 1)
ax2 = fig.add_subplot(2, 1, 2)

ax = fig.add_subplot(1, 1, 1, frameon=False)
plt.tick_params(labelcolor='none', top=False, bottom=False, left=False, right=False)
ax.set_ylabel("Depth [km]")
ax2.set_xlabel("Horizontal offset [km]")
ax1.set_xticklabels([])

for solver, ax, panel in (
    (solver_dg, ax1, f"(a)"), 
    (solver_ug, ax2, f"(b)"),
):
    ax.text(-0.025, 1.05, panel, va="bottom", ha="right", transform=ax.transAxes)
    qmesh = ax.pcolormesh(
        solver.vv.nodes[:,:,0,0], 
        solver.vv.nodes[:,:,0,1], 
        solver.vv.values[:,:,0],
        cmap=plt.get_cmap("jet")
    )
    ax.contour(
        solver.tt.nodes[:,:,0,0], 
        solver.tt.nodes[:,:,0,1], 
        solver.tt.values[:,:,0],
        colors="k",
        linestyles="--",
        linewidths=1,
        levels=np.arange(0, solver.tt.values.max(), 0.25)
    )
    ax.scatter(
        solver.vv.nodes[src_idx + (0,)],
        solver.vv.nodes[src_idx + (1,)],
        marker="*",
        facecolor="w",
        edgecolor="k",
        s=256
    )
    ax.invert_yaxis()
cbar = fig.colorbar(qmesh, ax=(ax1, ax2))
cbar.set_label("Velocity [km/s]")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-263-3539b8c22392>:18: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  qmesh = ax.pcolormesh(


In [264]:
p1=bp.figure()
p1.circle(df_ice.x, df_ice.t)
p1.circle(df_water.x, df_water.t)
p1.circle(np.linspace(-2560, 2560, num=512), solver_ug.tt.values[:,0,0])
#p1.circle(np.linspace(-2560, 2560, num=512), xx)

bp.show(p1)

In [180]:
plt.close("all")
vel=watermod
fig = plt.figure(figsize=(6, 2.5))

ax1 = fig.add_subplot(1, 1, 1)
#ax2 = fig.add_subplot(2, 1, 2)

ax = fig.add_subplot(1, 1, 1, frameon=False)
plt.tick_params(labelcolor='none', top=False, bottom=False, left=False, right=False)
ax.set_ylabel("Depth [km]")
ax1.set_xlabel("Horizontal offset [km]")
ax1.set_xticklabels([])

ax.text(-0.025, 1.05, panel, va="bottom", ha="right", transform=ax.transAxes)
qmesh = ax.pcolormesh(
    vel.nodes[:,:,0,0], 
    vel.nodes[:,:,0,1], 
    vel.values[:,:,0],
    cmap=plt.get_cmap("jet"),
    shading="auto"
)
#ax1.scatter([-2.56, 2.56], [0.7, 0.8])

ax1.invert_yaxis()
cbar = fig.colorbar(qmesh, ax=(ax1,))
cbar.set_label("Velocity [km/s]")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [101]:
fig1 = plt.figure()
axx = fig1.add_subplot()
axx.scatter(np.arange(512), solver_ug.tt.values[:,0])
#axx.scatter(np.linspace(0,1,512),solver_ug.tt.values[:,0,0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [149]:
np.arange(len(waterlay))

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [73]:
solver_ug.tt.values[0]

array([[0.53486298],
       [0.5327554 ],
       [0.53065984],
       [0.52857553],
       [0.5265021 ],
       [0.52443946],
       [0.52238768],
       [0.52034689],
       [0.51831729],
       [0.51629904],
       [0.51429231],
       [0.51229726],
       [0.51031403],
       [0.50834279],
       [0.50638373],
       [0.50443704],
       [0.50250294],
       [0.50058165],
       [0.49867342],
       [0.49677847],
       [0.49489705],
       [0.49302936],
       [0.49117559],
       [0.48933593],
       [0.48751049],
       [0.4856994 ],
       [0.48390273],
       [0.48212051],
       [0.48035275],
       [0.47859943],
       [0.47686047],
       [0.47513578],
       [0.47342521],
       [0.4717286 ],
       [0.47004573],
       [0.46837635],
       [0.46672017],
       [0.46507686],
       [0.46344605],
       [0.46182736],
       [0.46022036],
       [0.45862458],
       [0.45703956],
       [0.45546481],
       [0.45389981],
       [0.45234407],
       [0.45079707],
       [0.449

In [17]:
np.linspace?

In [285]:
solver_ug.tt.max_coords

array([2.55 , 1.023, 0.   ])

In [288]:
np.arange(solver_ug.tt.min_coords[0], solver_ug.tt.max_coords[0], .010)

array([-2.56000000e+00, -2.55000000e+00, -2.54000000e+00, -2.53000000e+00,
       -2.52000000e+00, -2.51000000e+00, -2.50000000e+00, -2.49000000e+00,
       -2.48000000e+00, -2.47000000e+00, -2.46000000e+00, -2.45000000e+00,
       -2.44000000e+00, -2.43000000e+00, -2.42000000e+00, -2.41000000e+00,
       -2.40000000e+00, -2.39000000e+00, -2.38000000e+00, -2.37000000e+00,
       -2.36000000e+00, -2.35000000e+00, -2.34000000e+00, -2.33000000e+00,
       -2.32000000e+00, -2.31000000e+00, -2.30000000e+00, -2.29000000e+00,
       -2.28000000e+00, -2.27000000e+00, -2.26000000e+00, -2.25000000e+00,
       -2.24000000e+00, -2.23000000e+00, -2.22000000e+00, -2.21000000e+00,
       -2.20000000e+00, -2.19000000e+00, -2.18000000e+00, -2.17000000e+00,
       -2.16000000e+00, -2.15000000e+00, -2.14000000e+00, -2.13000000e+00,
       -2.12000000e+00, -2.11000000e+00, -2.10000000e+00, -2.09000000e+00,
       -2.08000000e+00, -2.07000000e+00, -2.06000000e+00, -2.05000000e+00,
       -2.04000000e+00, -

In [297]:
ofs=np.linspace(-2.56, 2.56, num=512, endpoint=False)

In [298]:
for i in range(len(ofs)):
    print(i, ofs[i])

0 -2.56
1 -2.5500000000000003
2 -2.54
3 -2.5300000000000002
4 -2.52
5 -2.5100000000000002
6 -2.5
7 -2.49
8 -2.48
9 -2.47
10 -2.46
11 -2.45
12 -2.44
13 -2.43
14 -2.42
15 -2.41
16 -2.4
17 -2.39
18 -2.38
19 -2.37
20 -2.36
21 -2.35
22 -2.34
23 -2.33
24 -2.3200000000000003
25 -2.31
26 -2.3
27 -2.29
28 -2.2800000000000002
29 -2.27
30 -2.2600000000000002
31 -2.25
32 -2.24
33 -2.23
34 -2.22
35 -2.21
36 -2.2
37 -2.19
38 -2.18
39 -2.17
40 -2.16
41 -2.15
42 -2.14
43 -2.13
44 -2.12
45 -2.11
46 -2.1
47 -2.09
48 -2.08
49 -2.0700000000000003
50 -2.06
51 -2.05
52 -2.04
53 -2.0300000000000002
54 -2.02
55 -2.01
56 -2.0
57 -1.99
58 -1.98
59 -1.9700000000000002
60 -1.96
61 -1.9500000000000002
62 -1.94
63 -1.9300000000000002
64 -1.92
65 -1.9100000000000001
66 -1.9
67 -1.8900000000000001
68 -1.88
69 -1.87
70 -1.8599999999999999
71 -1.85
72 -1.84
73 -1.83
74 -1.82
75 -1.81
76 -1.8
77 -1.79
78 -1.78
79 -1.77
80 -1.76
81 -1.75
82 -1.74
83 -1.73
84 -1.7200000000000002
85 -1.71
86 -1.7000000000000002
87 -1.69
88